# Investments Portfolio

In [13]:
!pip install alpha_vantage
!pip install openpyxl
!pip install python-dotenv

In [9]:
import os
from dotenv import load_dotenv
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Load variables from .env file
load_dotenv()

# Get the API key securely
api_key = os.getenv("ALPHAVANTAGE_API_KEY")
ts = TimeSeries(key=api_key, output_format='pandas')

In [6]:
portfolio = pd.read_excel('portfolio.xlsx')
display(portfolio)

,Ativos,Início
0,ITUB4,10000
1,ABEV3,10000
2,VALE3,10000
3,BBDC3,10000


In [ ]:
# Search for the stock symbol
print('Searching for STOCKS')
print(ts.get_symbol_search('itub'))

# Get daily adjusted data for the stock
data, meta_data = ts.get_daily(symbol='ITUB4.SAO', outputsize='full')
display(data)

Searching for STOCKS
(                                1. symbol  \
date                                        
1970-01-01 00:00:00.000000000        ITUB   
1970-01-01 00:00:00.000000001  ITUB3F.SAO   
1970-01-01 00:00:00.000000002  ITUB4F.SAO   
1970-01-01 00:00:00.000000003      ITUBCK   
1970-01-01 00:00:00.000000004   ITUB3.SAO   
1970-01-01 00:00:00.000000005   ITUB4.SAO   

                                                                         2. name  \
date                                                                               
1970-01-01 00:00:00.000000000                         Itau Unibanco Holding S.A.   
1970-01-01 00:00:00.000000001                                             ITUB3F   
1970-01-01 00:00:00.000000002                                             ITUB4F   
1970-01-01 00:00:00.000000003  Turnaround Strategy Portfolio 20234 Invesco Un...   
1970-01-01 00:00:00.000000004                         Itaú Unibanco Holding S.A.   
1970-01-01 00:00:00.000000005

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2025-09-26,38.5400,38.8600,38.4900,38.6700,12414500.0
2025-09-25,38.8100,39.0200,38.4700,38.4800,14705600.0
2025-09-24,39.0600,39.1400,38.7100,38.7900,18168400.0
2025-09-23,38.4700,39.2400,38.4700,39.0500,16437000.0
2025-09-22,38.7300,38.8600,38.2800,38.4400,17791300.0
...,...,...,...,...,...
2005-01-07,393.0208,397.9968,391.0083,393.4993,129146796.0
2005-01-06,393.9976,399.0132,391.1010,393.9976,372692145.0
2005-01-05,397.9968,400.9995,395.0008,399.0000,863768854.0


In [26]:
stock_data = {}
date = '2021-01-05'


for stock in portfolio['Ativos']:
    print(f'Fetching data for {stock}')
    data, meta_data = ts.get_daily(symbol=f'{stock}.SAO', outputsize='full')
    stock_data[stock] = data
    
    portfolio.loc[portfolio['Ativos'] == stock, 'Qntd'] = portfolio.loc[portfolio['Ativos'] == stock, 'Início'].values / data.loc[date, '4. close'].values
    
display(portfolio)
display(stock_data['ITUB4'])



Fetching data for ITUB4


ValueError: We have detected your API key as JKQDVFAKFH4GVB77 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.